# Gateway Quickstart Integration Test

This notebook tests the complete Gateway setup flow from the quickstart guide.

**Prerequisites:**
- AWS credentials configured
- IAM permissions for Gateway, Lambda, IAM, and Cognito
- Bedrock model access (Claude Sonnet 3.7 or similar)

**What this notebook does:**
1. Creates an OAuth authorizer with Cognito
2. Creates a Gateway with MCP support
3. Adds a Lambda target with test tools
4. Tests the Gateway with an AI agent
5. Cleans up all resources

In [ ]:
# Configuration
REGION = "us-west-2"  # Change to your preferred region
MODEL_ID = "anthropic.claude-3-7-sonnet-20250219-v1:0"  # Change if needed

print(f"Region: {REGION}")
print(f"Model: {MODEL_ID}")

## Step 1: Initialize Gateway Client

In [ ]:
import json
import logging
import time

from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

# Initialize client
client = GatewayClient(region_name=REGION)
client.logger.setLevel(logging.INFO)

print("✅ Gateway client initialized")

## Step 2: Create OAuth Authorizer with Cognito

In [ ]:
print("Creating OAuth authorization server...")
cognito_response = client.create_oauth_authorizer_with_cognito("TestGateway")

print("\n✅ Authorization server created")
print(f"Client ID: {cognito_response['client_info']['client_id'][:20]}...")
print(f"User Pool ID: {cognito_response['client_info']['user_pool_id']}")

## Step 3: Create Gateway

In [ ]:
print("Creating Gateway...")
gateway = client.create_mcp_gateway(
    name=None,  # Auto-generated
    role_arn=None,  # Auto-created
    authorizer_config=cognito_response["authorizer_config"],
    enable_semantic_search=True,
)

print("\n✅ Gateway created")
print(f"Gateway ID: {gateway['gatewayId']}")
print(f"Gateway URL: {gateway['gatewayUrl']}")
print(f"Gateway ARN: {gateway['gatewayArn']}")

## Step 4: Fix IAM Permissions and Wait for Propagation

In [ ]:
print("Fixing IAM permissions...")
client.fix_iam_permissions(gateway)

print("⏳ Waiting 30s for IAM propagation...")
time.sleep(30)
print("✅ IAM permissions configured")

## Step 5: Add Lambda Target

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

client = GatewayClient(region_name=REGION)
print("Adding Lambda target...")
lambda_target = client.create_mcp_gateway_target(
    gateway=gateway,
    name=None,  # Auto-generated
    target_type="lambda",
    target_payload=None,  # Auto-created test Lambda
    credentials=None,
)

print("\n✅ Lambda target added")
print(f"Target ID: {lambda_target['targetId']}")
print(f"Target ARN: {lambda_target.get('targetArn', 'N/A')}")

## Step 6: Save Configuration

In [ ]:
config = {
    "gateway_url": gateway["gatewayUrl"],
    "gateway_id": gateway["gatewayId"],
    "gateway_arn": gateway["gatewayArn"],
    "region": REGION,
    "client_info": cognito_response["client_info"],
    "target_id": lambda_target["targetId"],
}

print("Configuration:")
print(json.dumps({k: v for k, v in config.items() if k != "client_info"}, indent=2))
print("\n✅ Configuration saved")

## Step 7: Get Access Token

In [ ]:
print("Getting access token...")
access_token = client.get_access_token_for_cognito(config["client_info"])

print(f"✅ Access token obtained (length: {len(access_token)})")
print(f"Token preview: {access_token[:50]}...")

## Step 8: Test Gateway with MCP Client

**Note:** This requires `strands-agents` package. Install with: `pip install strands-agents`

In [ ]:
try:
    from mcp.client.streamable_http import streamablehttp_client
    from strands.tools.mcp.mcp_client import MCPClient

    def create_streamable_http_transport(mcp_url: str, access_token: str):
        return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})

    def get_full_tools_list(mcp_client):
        """Get all tools with pagination support"""
        more_tools = True
        tools = []
        pagination_token = None
        while more_tools:
            tmp_tools = mcp_client.list_tools_sync(pagination_token=pagination_token)
            tools.extend(tmp_tools)
            if tmp_tools.pagination_token is None:
                more_tools = False
            else:
                more_tools = True
                pagination_token = tmp_tools.pagination_token
        return tools

    print("Testing MCP connection...")
    mcp_client = MCPClient(lambda: create_streamable_http_transport(config["gateway_url"], access_token))

    with mcp_client:
        tools = get_full_tools_list(mcp_client)
        print("\n✅ MCP connection successful")
        print(f"Available tools: {[tool.tool_name for tool in tools]}")
        print(f"Total tools: {len(tools)}")

except ImportError:
    print("⚠️  strands-agents not installed. Skipping MCP client test.")
    print("   Install with: pip install strands-agents")
except Exception as e:
    print(f"❌ MCP client test failed: {e}")

## Step 9: Test with AI Agent (Optional)

**Note:** This requires both `strands-agents` and Bedrock model access.

In [ ]:
try:
    from mcp.client.streamable_http import streamablehttp_client
    from strands import Agent
    from strands.models import BedrockModel
    from strands.tools.mcp.mcp_client import MCPClient

    print(f"Creating agent with model: {MODEL_ID}")

    # Setup Bedrock model
    bedrockmodel = BedrockModel(
        inference_profile_id=MODEL_ID,
        streaming=True,
    )

    # Setup MCP client
    mcp_client = MCPClient(lambda: create_streamable_http_transport(config["gateway_url"], access_token))

    with mcp_client:
        tools = get_full_tools_list(mcp_client)

        # Create agent
        agent = Agent(model=bedrockmodel, tools=tools)

        # Test query
        test_query = "What's the weather in Seattle?"
        print(f"\nTest query: {test_query}")
        print("\n🤔 Agent thinking...\n")

        response = agent(test_query)
        print(f"Agent response: {response.message.get('content', response)}")
        print("\n✅ Agent test successful")

except ImportError:
    print("⚠️  strands-agents not installed. Skipping agent test.")
    print("   Install with: pip install strands-agents")
except Exception as e:
    print(f"❌ Agent test failed: {e}")
    import traceback

    traceback.print_exc()

## Step 10: Test Gateway Update (Policy Engine)

Test updating the gateway configuration (without actually attaching a policy engine).

In [ ]:
print("Testing gateway update...")

# Test update with description
try:
    updated_gateway = client.update_gateway(
        gateway_identifier=config["gateway_id"],
        description="Updated via integration test",
    )
    print("✅ Gateway update successful")
    print(f"Description: {updated_gateway.get('description', 'N/A')}")
except Exception as e:
    print(f"❌ Gateway update failed: {e}")

## Step 11: Cleanup Resources

**Important:** This will delete all created resources. Make sure you're done testing!

In [ ]:
print("Starting cleanup...")
try:
    client.cleanup_gateway(config["gateway_id"], config["client_info"])
    print("✅ Cleanup complete!")
    print("   - Gateway deleted")
    print("   - Lambda function deleted")
    print("   - IAM roles deleted")
    print("   - Cognito resources deleted")
except Exception as e:
    print(f"❌ Cleanup failed: {e}")
    print("   You may need to manually delete resources")

## Summary

This notebook tested:
- ✅ Gateway client initialization
- ✅ OAuth authorizer creation with Cognito
- ✅ Gateway creation with MCP support
- ✅ IAM permissions configuration
- ✅ Lambda target creation
- ✅ Access token generation
- ✅ MCP client connection (if strands-agents installed)
- ✅ AI agent integration (if strands-agents installed)
- ✅ Gateway update functionality

**Next Steps:**
- Test with custom Lambda functions
- Add OpenAPI targets
- Test with Policy Engine integration
- Test cleanup functionality